In [1]:
import os
os.chdir("../")
%pwd

'd:\\DataScience\\Thyroid-disease-prediction'

In [2]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModulEvaluationConfig:
    root_dit: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [3]:
from Thyroid_Disease.constants import *
from Thyroid_Disease.utils.common import read_yaml, create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evalution_config(self) -> ModulEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN


        create_directories([config.root_dir])

        model_evaluation_config = ModulEvaluationConfig(
            root_dit=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/rawatshubham09/Thyroid-disease-prediction.mlflow",
        )

        return model_evaluation_config

In [29]:
import os
import pandas as pd
from sklearn.metrics import recall_score, confusion_matrix, f1_score, accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [35]:
class ModelEvaluation:
    def __init__(self, config: ModulEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        recall = recall_score(actual,pred, average="weighted")
        #confusion_mat = confusion_matrix(actual, pred)
        f1_s = f1_score(actual,pred,average="weighted")
        accuracy = accuracy_score(actual,pred)

        return recall, f1_s, accuracy
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme # https:

        with mlflow.start_run():
            pred_y = model.predict(test_x)

            (recall, f1_sc, accuracy) = self.eval_metrics(test_y, pred_y)

            # saving metrics as local
            scores = {"Recall_score": recall, "f1_score": f1_sc, "Accuracy":accuracy}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params) #XGBParams
            mlflow.log_metric("Recall_Score", recall)
            mlflow.log_metric("F1_Score",f1_sc)
            mlflow.log_metric("Accuracy",accuracy)

            if tracking_url_type_store != "file":
                # check if it is a local directory or not
                mlflow.sklearn.log_model(model,"model",registered_model_name="XGBoost")
            else:
                mlflow.sklearn.log_model(model,"model")





In [36]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evalution_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-06 11:58:28,591 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-04-06 11:58:28,594 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-04-06 11:58:28,596 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-04-06 11:58:28,601 : INFO : common : created directory at: artifacts]
[2024-04-06 11:58:28,602 : INFO : common : created directory at: artifacts/model_evaluation]
[2024-04-06 11:58:28,742 : INFO : common : json file saved at: artifacts\model_evaluation\metrics.json]
